In [1]:
import sys
import os
from elasticsearch import Elasticsearch
from zipfile import ZipFile
import re
import pandas as pd
import json
import requests
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import nltk
import pickle
import pytrec_eval
nltk.download('stopwords')
nltk.download('punkt')
from elasticsearch import NotFoundError

es = Elasticsearch()

[nltk_data] Downloading package stopwords to /home/katya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/katya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
input_dir = 'data/docs'
output_dir = 'res'

In [3]:
def strip_punct(s):
    s = re.sub('[^A-Za-z0-9]', ' ', s)
    s = s.lower()
    return " ".join(s.split())

## Create test-index with 110 docs for each topic

In [4]:
with open('bulk_data_json.txt') as f:
    bulk_data = json.load(f)

In [5]:
bulk_data[1]

{'query': 'which and has and more and caffeine and coffee and or and tea',
 'title': 'Schabell: Caffee vs <em>Tea</em>, <em>which</em> <em>has</em> <em>more</em> <em>caffeine</em>',
 'title_lem': 'schabell caffee v tea caffeine',
 'num': '31',
 'uuid': 'c90d618b-3dfd-55b9-bbdc-07cdeb747b69',
 'score': 2512.4922,
 'document': 'Caffee vs Tea, which has more caffeine\nThis is an off the wall posting about a topic that has been running around the workplace at about every IT job I have had. Coffee drinkers get headaches in the weekend from too much caffeine and think switching to tea will help. I finally sat down and did a bit of research on this and here are my findings (a hint, the tea drinkers are partially right):\nAt the Indiana University Library you can see the details, but the summary is that "..an average pound of tea has the equivalent caffeine content of two average pounds of coffee." As one does not eat coffee or tea by the bean or leaf, this means nothing.\nIf we look at the re

In [4]:
def return_arguments(doc_raw, model):
    url = 'https://demo.webis.de/targer-api/classify'+model
    #print(url)
    #print(doc_raw)
    res = requests.post(url, json=doc_raw)
    if res.status_code == 200:
#print(res)
        res = res.json()
        args = []
        for x in res:
            arg = []
            for el in x:
                if el['label']!='O' and float(el['prob'])>0.99:
                    arg.append(el['token'])
            s = ' '.join(arg)
            args.append(s)
        arguments = ' '.join(args)
        arguments = word_tokenize(strip_punct(arguments).lower())
        tokens_without_sw = [word for word in arguments if not word in stopwords.words()]
        lemmatizer = WordNetLemmatizer()
        lemmatized_output_0 = ([lemmatizer.lemmatize(w,pos="n") for w in tokens_without_sw])
        arguments = ' '.join(([lemmatizer.lemmatize(w,pos="v") for w in lemmatized_output_0]))
    else:
        #print('EMPTY')
        #print(res)
        arguments = ''
    return arguments

In [5]:
count = 0
c=0
t = 1
url = 'https://demo.webis.de/targer-api/classifyCombo'
headers = {'accept': 'application/json', 'Content-Type': 'text/plain'}
bulk_data = []
#extract docs from zip-files
for filename in os.listdir(input_dir):
    if filename.endswith(".zip"):
        with ZipFile((input_dir+"/{}").format(filename), 'r') as zip:
            zip.extractall(input_dir)
            
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        name = re.split('_|\.', filename)
        num = next(obj for obj in name if obj.isdigit())
        with open((input_dir+"/{}").format(filename), "r") as f:
            topic = json.load(f)
            topic['title'] = strip_punct(topic['title']).lower()
            print(filename)
            print(topic['title'])
            print(t)
            t+=1
            for n in topic["documents"]:
                print(len(n))
                #print(count)
                if count<10:
                    count+=1
                    for doc in n:
                        try:
                            doc_raw = doc['document'].rstrip('\n')
                            doc_raw = doc_raw.rstrip('\\n')
                            
                            doc['lem'] = word_tokenize(strip_punct(doc_raw).lower())
                            tokens_without_sw = [word for word in doc['lem'] if not word in stopwords.words()]
                            lemmatizer = WordNetLemmatizer()
                            lemmatized_output_0 = ([lemmatizer.lemmatize(w,pos="n") for w in tokens_without_sw])
                            doc['lem'] = ' '.join(([lemmatizer.lemmatize(w,pos="v") for w in lemmatized_output_0]))
                            doc['lem'] = doc['lem'].rstrip()
                            
                            title_lem = strip_punct(doc['title'].lower().rstrip())
                            title_lem = word_tokenize(title_lem)
                            tokens_without_sw = [word for word in title_lem if not word in stopwords.words()]
                            lemmatizer = WordNetLemmatizer()
                            lemmatized_output_0 = ([lemmatizer.lemmatize(w,pos="n") for w in tokens_without_sw])
                            title_lem = ' '.join(([lemmatizer.lemmatize(w,pos="v") for w in lemmatized_output_0]))
                            
                            #doc_raw = doc['document'].rstrip('\n')
                            #doc_raw = doc_raw.rstrip('\\n')
                            
                            
                            combo = return_arguments(doc_raw, 'Combo')
                            
                            es = return_arguments(doc_raw, 'ES')
                            
                            es_dep = return_arguments(doc_raw, 'ES_dep')
                            
                            ibm = return_arguments(doc_raw, 'IBM')
                            
                            new_pe = return_arguments(doc_raw, 'NewPE')
                            
                            new_wd = return_arguments(doc_raw, 'NewWD')
                            
                            wd = return_arguments(doc_raw, 'WD')
                            
                            wd_dep = return_arguments(doc_raw, 'WD_dep')
                        
                            
                            
                            
                            b = {
                                    'query': topic['title'],
                                    'title': doc['title'],
                                    'title_lem': title_lem,
                                    'num': num,
                                    'uuid': doc['uuid'],
                                    'score': doc['score'],
                                    'document': doc['document'],
                                    'lem': doc['lem'],
                                    'combo': combo,
                                    'es': es,
                                    'es_dep': es_dep,
                                    'ibm': ibm,
                                    'new_pe': new_pe,
                                    'new_wd': new_wd,
                                    'wd': wd,
                                    'wd_dep': wd_dep
                                }
                            #es.index(index='test_index',doc_type='doc',id=doc['trec_id'],body=b)
                            templ = {'index': {'_index': 'test_index', 
                                           '_type': 'doc', 
                                           '_id': doc['trec_id']}}
                            bulk_data.append(templ)
                            bulk_data.append(b)

                            c+=1
                        except KeyError:
                            pass
                else:
                    break
            count=0


docs_for_topic_31.txt
which and has and more and caffeine and coffee and or and tea
1
10
10
10
10
10
10
10
10
10
10
10
docs_for_topic_27.txt
which and one and is and better and an and electric and stove and or and a and gas and stove
2
10
10
10
10
10
10
10
10
10
10
10
docs_for_topic_9.txt
why and is and linux and better and than and windows
3
10
10
10
10
10
10


KeyboardInterrupt: 

In [ ]:
with open("bulk_data_json_all_models.txt", "w") as f:
    json.dump(bulk_data,f)

## Create different runs

In [129]:
f = open("data/topics_lemmatized.txt", 'rb')
queries_lem = pickle.load(f)
f.close()

In [130]:
def search(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1

    for i in q:
        for x in search_param['query']['bool']['should']:
            if type(i) == dict:
                for key in x['match']:
                    x['match'][key] = i['title']
            else:
                for key in x['match']:
                    x['match'][key] = i

    
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [6]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "title_lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_title', queries_lem, "test_index")

In [7]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "lem": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_doc', queries_lem, "test_index")

In [8]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [ {
         'match': {
            "args": ''
        }}
      ]
    }
    }
}

search(search_param, 'run_arg', queries_lem, "test_index")

In [9]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_arg', queries_lem, "test_index")

In [10]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "lem": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc', queries_lem, "test_index")

In [11]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_doc_arg', queries_lem, "test_index")

In [12]:
search_param = {
    
    'size': 50,
    "query": {
    "bool": {
      "should": [
        {
          "match": {
            "lem": ""
          }
        },
        {
          "match": {
            "title_lem": ""
          }
        },
        {
          "match": {
            "args": ""
          }
        }
      ]
    }
  }
}

search(search_param, 'run_title_doc_arg', queries_lem, "test_index")

## Add synonyms

In [13]:
def search_extended(search_param, filename, q, ind):
    
    qid = []
    Q0 = []
    doc = []
    rank = []
    score = []
    tag = []

    num=1
#['bool']['should']
    for idx, row in q.iterrows():
            
            #for key in x['match']:
                #x['match'][key] = row['query']
        for x in search_param['query']['bool']['should']:
            for key in x["match"]:
                #print(key)
                if "boost" in x['match'][key]:
                    x['match'][key]["query"] = row['query']
                else:
                    x['match'][key] = row['syn']
        #print(search_param)
        response = es.search(index=ind, body=search_param)
        r = 1
        for x in response['hits']['hits']:
            qid.append(num)
            Q0.append('Q0')
            doc.append(x['_id'])
            rank.append(r)
            score.append(x['_score'])
            tag.append('uh-t2-thor')
            r+=1
        num+=1
    
   
    qrels = {'qid': qid, 'Q0': Q0, 'doc': doc, 'rank':rank, 'score':score, 'tag':tag}

    df = pd.DataFrame(qrels)
    df.to_csv((output_dir+'/'+filename+'.txt'), sep = ' ', index = False, header = False)

In [14]:
df_syn = pd.read_csv('data/q_for_syn.tsv', sep = '\t')

In [15]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_syn', df_syn, "test_index")

In [16]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_syn', df_syn, "test_index")

In [17]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_args_syn', df_syn, "test_index")

In [18]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_arg_syn', df_syn, "test_index")

In [19]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_syn', df_syn, "test_index")

In [20]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {""}}},
                    {"match": {"args": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_doc_arg_syn', df_syn, "test_index")

In [21]:
search_param = {
    
    'size': 50, 
    "query": {
            "bool":{
                "should": [
                    {"match": {"title_lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"args": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"lem": {
                        "query":"",
                    "boost":5}}},
                    {"match": {"title_lem": {""}}},
                    {"match": {"args": {""}}},
                    {"match": {"lem": {""}}}
                ]
            }
          }
      
}
search_extended(search_param, 'run_title_doc_arg_syn', df_syn, "test_index")

## Evaluate and compare results

In [22]:
def evaluate_run(run_path, qrel_path):
    run = pd.read_csv(run_path, sep = ' ', names = ['topic','Q0','id','rank','score','team'])

    qrel = pd.read_csv(qrel_path, sep = ' ', names = ['topic','Q0','id','relevance'])

    qrels = {}
    for i in range(1,51):
        qrels[str(i)] = {}
    
    for idx, row in qrel.iterrows():
        qrels[str(row['topic'])][row['id']] = row['relevance']
    
    runs = {}
    for i in range(1,51):
        runs[str(i)] = {}
    
    for idx, row in run.iterrows():
        runs[str(row['topic'])][row['id']] = row['score']

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrels,{'map_cut', 'ndcg_cut', 'recall', 'P'})

    res = evaluator.evaluate(runs)

    results = {}
    recall = {}
    pres = {}
    for key in res:
        results[key] = res[key]['ndcg_cut_10']
        recall[key] = res[key]['recall_10']
        pres[key] = res[key]['P_10']
        #print(res[key])
    
    filename = re.split('\.', sys.argv[1])
    with open((filename[0]+'_evaluated.json'), 'w') as f:
        json.dump(results, f)
        
    s = sum(results.values())/50
    r = sum(recall.values())/50
    p = sum(pres.values())/50
    if r+p != 0:
        f1 = (2*(r*p)/(r+p))
    else: 
        f1 = 0
    '''
    print("Average ndcg_cut_10: ", s)
    print("Average recall_10: ", r)
    print("Av. precision: ", p)
    print("F1-score: ",f1)
    '''    
    return s, r, p, f1


In [23]:
run = []
ndcg = []
rec = []
prec = []
f1s = []

for filename in os.listdir(output_dir):
    if filename.endswith(".txt"):
        #print(filename)
        s, r, p, f1 = evaluate_run((output_dir+"/{}").format(filename), 'touche2020-task2-relevance-withbaseline(1).qrels')
        run.append(filename)
        ndcg.append(s)
        rec.append(r)
        prec.append(p)
        f1s.append(f1)

In [24]:
df = pd.DataFrame({'run':run,'ndcg_cut10':ndcg, 'recall_10':rec, 'precision':prec, 'f1-score':f1s})

### All results

In [25]:
df.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
11,run_title_doc_arg_syn.txt,0.378309,0.234088,0.358,0.283078
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [26]:
df.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
11,run_title_doc_arg_syn.txt,0.378309,0.234088,0.358,0.283078
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [27]:
df.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
2,run_title.txt,0.372117,0.232918,0.360,0.282840
11,run_title_doc_arg_syn.txt,0.378309,0.234088,0.358,0.283078
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [28]:
df.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
11,run_title_doc_arg_syn.txt,0.378309,0.234088,0.358,0.283078
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


### Results without doc

In [47]:
df_p = df[~df.run.str.contains("doc")]

In [48]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
8,run_args_syn.txt,0.269069,0.180429,0.262,0.213695
9,run_arg.txt,0.267192,0.179525,0.262,0.213059


In [49]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
8,run_args_syn.txt,0.269069,0.180429,0.262,0.213695
9,run_arg.txt,0.267192,0.179525,0.262,0.213059


In [50]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
9,run_arg.txt,0.267192,0.179525,0.262,0.213059
8,run_args_syn.txt,0.269069,0.180429,0.262,0.213695


In [51]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
1,run_title_arg_syn.txt,0.384752,0.252236,0.376,0.301927
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
8,run_args_syn.txt,0.269069,0.180429,0.262,0.213695
9,run_arg.txt,0.267192,0.179525,0.262,0.213059


### Results for pipeline

In [42]:
df_p = df[~df.run.str.contains("arg")]
df_p = df_p[~df_p.run.str.contains("syn")]

In [43]:
df_p.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [44]:
df_p.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [45]:
df_p.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [46]:
df_p.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


### Results for synonyms

In [30]:
df_syn = df[~df.run.str.contains("arg")]

In [31]:
df_syn.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [33]:
df_syn.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [34]:
df_syn.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


In [35]:
df_syn.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
10,run_title_doc_syn.txt,0.435423,0.276490,0.426,0.335335
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
2,run_title.txt,0.372117,0.232918,0.360,0.282840
13,run_title_syn.txt,0.350643,0.222208,0.340,0.268764
4,run_doc_syn.txt,0.318743,0.196126,0.296,0.235929
7,run_doc.txt,0.317627,0.195354,0.296,0.235369


### Results for arguments

In [37]:
df_arg = df[~df.run.str.contains("syn")]

In [38]:
df_arg.sort_values(by = ['ndcg_cut10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369
6,run_doc_arg.txt,0.285701,0.189667,0.278,0.225492
9,run_arg.txt,0.267192,0.179525,0.262,0.213059


In [39]:
df_arg.sort_values(by = ['recall_10'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369
6,run_doc_arg.txt,0.285701,0.189667,0.278,0.225492
9,run_arg.txt,0.267192,0.179525,0.262,0.213059


In [40]:
df_arg.sort_values(by = ['precision'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369
6,run_doc_arg.txt,0.285701,0.189667,0.278,0.225492
9,run_arg.txt,0.267192,0.179525,0.262,0.213059


In [41]:
df_arg.sort_values(by = ['f1-score'], ascending = False)

,run,ndcg_cut10,recall_10,precision,f1-score
3,run_title_doc.txt,0.442337,0.271286,0.424,0.330871
12,run_title_arg.txt,0.386302,0.250989,0.374,0.300389
0,run_title_doc_arg.txt,0.384614,0.236975,0.362,0.286439
2,run_title.txt,0.372117,0.232918,0.360,0.282840
7,run_doc.txt,0.317627,0.195354,0.296,0.235369
6,run_doc_arg.txt,0.285701,0.189667,0.278,0.225492
9,run_arg.txt,0.267192,0.179525,0.262,0.213059
